In [7]:

import sys
!{sys.executable} -m pip install biopandas

In [3]:
import pandas
import numpy as np
import sys,argparse
import time
import os
from biopandas.mol2 import PandasMol2
from Bio.PDB import *
from biopandas.pdb import PandasPdb

In [4]:
pmol = PandasMol2().read_mol2('site.mol2')
#contains atoms
pmol.df.head(3)


,atom_id,atom_name,x,y,z,atom_type,subst_id,subst_name,charge
0,1,C,5.662,-11.097,73.632,C.2,1,HIS16,0.0
1,2,O,6.578,-11.742,73.127,O.2,1,HIS16,0.0
2,3,CA,5.810,-10.555,75.059,C.3,1,HIS16,0.0


In [5]:
ppdb_df =  PandasPdb().read_pdb('1i9c.pdb')

In [6]:
atom_df = ppdb_df.df['ATOM']
atom_df.head()

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,MET,,A,1,,...,14.161,-9.448,46.656,1.0,30.61,,,N,NaN,672
1,ATOM,2,,CA,,MET,,A,1,,...,15.242,-10.147,47.340,1.0,30.21,,,C,NaN,673
2,ATOM,3,,C,,MET,,A,1,,...,15.101,-11.669,47.282,1.0,35.55,,,C,NaN,674
3,ATOM,4,,O,,MET,,A,1,,...,14.011,-12.216,47.430,1.0,28.46,,,O,NaN,675
4,ATOM,5,,CB,,MET,,A,1,,...,15.291,-9.693,48.797,1.0,24.49,,,C,NaN,676


In [7]:
atom_df.columns

Index(['record_name', 'atom_number', 'blank_1', 'atom_name', 'alt_loc',
       'residue_name', 'blank_2', 'chain_id', 'residue_number', 'insertion',
       'blank_3', 'x_coord', 'y_coord', 'z_coord', 'occupancy', 'b_factor',
       'blank_4', 'segment_id', 'element_symbol', 'charge', 'line_idx'],
      dtype='object')

In [8]:
#read P2rank predictions
p2rank_pred_df = pandas.read_csv('1i9c.pdb_predictions.csv')


In [9]:
p2rank_pred_df.head()

,name,rank,score,probability,sas_points,surf_atoms,center_x,center_y,center_z,residue_ids,surf_atom_ids
0,pocket1,1,116.07,0.994,397,191,7.3103,-4.4831,77.3370,A_117 A_118 A_120 A_121 A_122 A_123 A_126 A_1...,92 101 103 106 108 110 115 116 117 118 119 12...
1,pocket2,2,106.64,0.993,361,183,5.5729,36.2659,87.8053,C_117 C_118 C_120 C_121 C_122 C_123 C_126 C_1...,4935 4963 4974 4977 4979 4981 4987 4988 4989 ...
2,pocket3,3,9.02,0.317,58,29,-19.2652,21.9690,88.5728,B_356 B_357 B_358 D_257 D_258 D_259 D_364 D_3...,3832 3837 3838 3840 3842 3845 3846 7945 7946 ...
3,pocket4,4,9.00,0.316,63,30,-6.7129,41.8543,96.6379,C_21 C_25 D_407 D_408 D_409 D_414 D_416 D_417...,5016 5017 5018 5020 5022 5026 5056 5058 9094 ...
4,pocket5,5,7.39,0.231,71,35,12.2077,16.1648,83.3353,B_301 B_302 B_305 B_328 B_329 B_338 B_339 B_3...,1767 3441 3444 3445 3446 3451 3474 3476 3628 ...


In [10]:
p2rank_pred_df.columns

Index(['name     ', '  rank', '   score', ' probability', ' sas_points',
       ' surf_atoms', '   center_x', '   center_y', '   center_z',
       ' residue_ids', ' surf_atom_ids'],
      dtype='object')

In [11]:
#for each pocket get from atom_df the corresponding coordinates
for index, row in p2rank_pred_df.iterrows():
    
    #get pocket atom ids
    ids = row[' surf_atom_ids'].split()
    print(len(ids))

    #get the corresponding coordinates
    pocket_atom_xyz = atom_df[atom_df['atom_number'].isin(ids)]
    print(pocket_atom_xyz)  
    
    #use row[name]for the output file
    out_file = row['name     ']+".pdb"
    
    #save the pdb file with the desired fields
    pocket_atom_xyz[['record_name', 'atom_number', 'blank_1', 'atom_name', 
       'residue_name', 'blank_2', 'chain_id', 'residue_number', 'insertion',
       'blank_3', 'x_coord', 'y_coord', 'z_coord', 'occupancy', 'b_factor',
       'blank_4', 'segment_id', 'element_symbol']].to_csv(out_file,index=False,header=False,sep=' ')
    


191
     record_name  atom_number blank_1 atom_name alt_loc residue_name blank_2  \
91          ATOM           92                 O                  SER           
100         ATOM          101               OD1                  ASP           
102         ATOM          103                 N                  CYS           
105         ATOM          106                 O                  CYS           
107         ATOM          108                SG                  CYS           
...          ...          ...     ...       ...     ...          ...     ...   
4789        ATOM         4791                CA                  GLY           
4790        ATOM         4792                 C                  GLY           
4791        ATOM         4793                 O                  GLY           
4797        ATOM         4799                CG                  LYS           
4799        ATOM         4801                CE                  LYS           

     chain_id  residue_number inser

25
     record_name  atom_number blank_1 atom_name alt_loc residue_name blank_2  \
3079        ATOM         3081               OD1                  ASN           
3080        ATOM         3082               ND2                  ASN           
3089        ATOM         3091                 N                  ILE           
3093        ATOM         3095                CB                  ILE           
3094        ATOM         3096               CG1                  ILE           
3095        ATOM         3097               CG2                  ILE           
3097        ATOM         3099                 N                  GLN           
3895        ATOM         3897                CA                  GLU           
3897        ATOM         3899                 O                  GLU           
3898        ATOM         3900                CB       A          GLU           
3900        ATOM         3902                CG       A          GLU           
3929        ATOM         3931        

     record_name  atom_number blank_1 atom_name alt_loc residue_name blank_2  \
5578        ATOM         5581                 O                  VAL           
5580        ATOM         5583               CG1                  VAL           
6467        ATOM         6471                 O                  ARG           
6476        ATOM         6480                CA                  ALA           
6488        ATOM         6492                CB                  VAL           
6489        ATOM         6493               CG1                  VAL           
6490        ATOM         6494               CG2                  VAL           
6496        ATOM         6500                 N                  LEU           
6501        ATOM         6505                CG                  LEU           
6502        ATOM         6506               CD1                  LEU           
6523        ATOM         6527                 O                  GLU           
6524        ATOM         6528           

     record_name  atom_number blank_1 atom_name alt_loc residue_name blank_2  \
4991        ATOM         4994                CB                  ALA           
5009        ATOM         5012               OD1                  ASN           
5017        ATOM         5020                CG       A          LYS           
5023        ATOM         5026                NZ       A          LYS           
5048        ATOM         5051               OD2                  ASP           
5135        ATOM         5138                 O                  ASN           
5137        ATOM         5140                CG                  ASN           
5139        ATOM         5142               ND2                  ASN           
5142        ATOM         5145                 C                  ILE           
5143        ATOM         5146                 O                  ILE           
5148        ATOM         5151                 N                  GLY           
5149        ATOM         5152           

     record_name  atom_number blank_1 atom_name alt_loc residue_name blank_2  \
3456        ATOM         3458                CD                  LYS           
3457        ATOM         3459                CE                  LYS           
3458        ATOM         3460                NZ                  LYS           
4795        ATOM         4797                 O                  LYS           
4804        ATOM         4806                 O                  GLY           
6637        ATOM         6641                 O                  GLY           
6639        ATOM         6643                CA                  GLY           
8631        ATOM         8635                CB                  LYS           
8633        ATOM         8637                CD                  LYS           
8634        ATOM         8638                CE                  LYS           
8635        ATOM         8639                NZ                  LYS           
8642        ATOM         8646           

     record_name  atom_number blank_1 atom_name alt_loc residue_name blank_2  \
1130        ATOM         1132                CA                  ASP           
1132        ATOM         1134                 O                  ASP           
1133        ATOM         1135                CB                  ASP           
1134        ATOM         1136                CG                  ASP           
1135        ATOM         1137               OD1                  ASP           
1136        ATOM         1138               OD2                  ASP           
1162        ATOM         1164               CD2                  PHE           
1164        ATOM         1166               CE2                  PHE           
1170        ATOM         1172                CB                  HIS           
1171        ATOM         1173                CG                  HIS           
1172        ATOM         1174               ND1                  HIS           
1174        ATOM         1176           

     record_name  atom_number blank_1 atom_name alt_loc residue_name blank_2  \
5915        ATOM         5919                 N                  MET           
5919        ATOM         5923                CB                  MET           
5922        ATOM         5926                CE                  MET           
5986        ATOM         5990               CZ2                  TRP           
6059        ATOM         6063               OE1                  GLN           
6088        ATOM         6092               OE1                  GLU           
6121        ATOM         6125               OE1                  GLN           
7163        ATOM         7167               OD2                  ASP           
7178        ATOM         7182                CZ                  ARG           
7424        ATOM         7428                CG                  LYS           
7425        ATOM         7429                CD                  LYS           
7427        ATOM         7431           

9
     record_name  atom_number blank_1 atom_name alt_loc residue_name blank_2  \
7968        ATOM         7972               NE2                  GLN           
8803        ATOM         8807               OE2                  GLU           
8822        ATOM         8826               CG1                  VAL           
8853        ATOM         8857               OE1                  GLU           
8854        ATOM         8858               OE2                  GLU           
9253        ATOM         9257                 O                  VAL           
9263        ATOM         9267                CD                  ARG           
9265        ATOM         9269                CZ                  ARG           
9422        ATOM         9426               OD1                  ASN           

     chain_id  residue_number insertion  ... x_coord  y_coord  z_coord  \
7968        D             260            ... -15.784   29.411   87.595   
8803        D             368            ... -20.

16
     record_name  atom_number blank_1 atom_name alt_loc residue_name blank_2  \
7667        ATOM         7671                CA                  PRO           
7669        ATOM         7673                 O                  PRO           
7689        ATOM         7693                CA                  GLY           
7690        ATOM         7694                 C                  GLY           
7691        ATOM         7695                 O                  GLY           
7699        ATOM         7703                 N                  LEU           
7700        ATOM         7704                CA                  LEU           
7707        ATOM         7711                 N                  VAL           
7710        ATOM         7714                 O                  VAL           
7711        ATOM         7715                CB                  VAL           
7713        ATOM         7717               CG2                  VAL           
7748        ATOM         7752        

     record_name  atom_number blank_1 atom_name alt_loc residue_name blank_2  \
7667        ATOM         7671                CA                  PRO           
7669        ATOM         7673                 O                  PRO           
7689        ATOM         7693                CA                  GLY           
7690        ATOM         7694                 C                  GLY           
7707        ATOM         7711                 N                  VAL           
7710        ATOM         7714                 O                  VAL           
7711        ATOM         7715                CB                  VAL           
7713        ATOM         7717               CG2                  VAL           
7748        ATOM         7752                OG       A          SER           
7915        ATOM         7919                 C                  GLY           
7916        ATOM         7920                 O                  GLY           
7920        ATOM         7924           

In [28]:


def main(argv):
    parser = argparse.ArgumentParser()
    parser.add_argument('input_toconvert',  
                        nargs='?'
                        )
    parser.add_argument('input_pdb',  
                        nargs='?'
                        )
    parser.add_argument('output',  
                        nargs='?'
                        )
    args = parser.parse_args()
    path_a = args.input_toconvert
    path_b = args.input_pdb
    path_out = args.output
    inter()
   
 
if __name__ == "__main__":
    main(sys.argv[1:])

usage: ipykernel_launcher.py [-h] [input_toconvert] [input_pdb] [output]
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


The overall layout of a Structure object follows the so-called SMCRA (Structure/Model/Chain/Residue/Atom) architecture:

    A structure consists of models
    A model consists of chains
    A chain consists of residues
    A residue consists of atoms


In [14]:
p = PDBParser()
structure = p.get_structure("1i9c", "1i9c.pdb")
#for model in structure:
#    for chain in model:
#        for residue in chain:
#            for atom in residue:
#                print(atom)

#There are also some shortcuts:

# Iterate over all atoms in a structure
for atom in structure.get_atoms():
    print(a.get_name()  # atom name (spaces stripped, e.g. 'CA')
    print(a.get_id()  # id (equals atom name)
    print(a.get_coord()  # atomic coordinates
    print(a.get_vector()  # atomic coordinates as Vector object
    print(a.get_bfactor()  # isotropic B factor
    print(a.get_occupancy()  # occupancy
    print(a.get_altloc()  # alternative location specifier
    print(a.get_sigatm()  # std. dev. of atomic parameters
    print(a.get_siguij()  # std. dev. of anisotropic B factor
    print(a.get_anisou()  # anisotropic B factor
   print( a.get_fullname()  # atom name (with spaces, e.g. '.CA.')

# Iterate over all residues in a model
#for residue in model.get_residues():
#    print(residue)


C:\ProgramData\Anaconda3\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 10415.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 10506.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 10562.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 10653.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 10709.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 10834.
  warnings.warn(
C:\Program

[14.161 -9.448 46.656]
[ 15.242 -10.147  47.34 ]
[ 15.101 -11.669  47.282]
[ 14.011 -12.216  47.43 ]
[15.291 -9.693 48.797]
[ 16.239 -10.444  49.707]
[ 15.677 -10.385  51.426]
[ 14.583 -11.805  51.482]
[ 16.229 -12.325  47.07 ]
[ 16.416 -13.766  47.101]
[ 15.922 -14.37   48.415]
[ 16.222 -13.856  49.49 ]
[ 17.894 -14.14   46.928]
[ 18.146 -15.141  45.809]
[ 19.525 -14.928  45.207]
[ 20.501 -15.466  45.776]
[ 19.613 -14.214  44.183]
[ 15.165 -15.462  48.389]
[ 14.651 -16.015  49.647]
[ 15.751 -16.679  50.485]
[ 16.601 -17.35   49.895]
[ 13.543 -17.02   49.364]
[ 12.897 -17.67   50.583]
[ 12.358 -19.053  50.213]
[ 10.882 -19.21   50.542]
[ 10.401 -20.552  50.082]
[ 15.681 -16.461  51.784]
[ 16.454 -16.988  52.881]
[ 15.551 -17.294  54.09 ]
[ 14.444 -16.766  54.203]
[ 17.555 -16.033  53.338]
[ 18.529 -15.567  52.276]
[ 19.381 -16.738  51.824]
[ 20.406 -16.258  50.79 ]
[ 21.431 -17.345  50.655]
[ 16.05  -18.146  54.976]
[ 15.341 -18.58   56.171]
[ 15.921 -17.997  57.447]
[ 17.118 -17.952  

[ 15.372 -10.79   76.286]
[12.391 -8.952 75.296]
[11.787 -7.697 75.703]
[11.84  -7.483 77.208]
[11.881 -6.338 77.645]
[10.309 -7.652 75.285]
[ 9.919 -7.403 73.835]
[10.715 -6.246 73.228]
[10.073 -8.633 72.98 ]
[11.81  -8.571 77.979]
[11.456 -8.49  79.39 ]
[12.591 -8.092 80.327]
[12.3   -7.875 81.498]
[10.798 -9.789 79.838]
[ 11.556 -11.052  80.056]
[ 12.774 -11.36   79.485]
[ 10.99  -12.023  80.886]
[ 13.412 -12.561  79.733]
[ 11.625 -13.216  81.14 ]
[ 12.843 -13.487  80.564]
[ 13.474 -14.684  80.805]
[13.808 -7.968 79.823]
[14.962 -7.508 80.526]
[15.9   -8.607 80.978]
[17.058 -8.274 81.203]
[15.416 -9.845 81.125]
[ 16.293 -10.906  81.641]
[ 17.09  -11.571  80.531]
[ 17.806 -12.545  80.772]
[ 15.463 -11.914  82.449]
[ 14.819 -11.285  83.675]
[ 13.989 -12.106  84.61 ]
[ 13.727 -11.646  85.725]
[ 13.511 -13.297  84.251]
[ 17.011 -11.045  79.301]
[ 17.783 -11.647  78.225]
[ 19.287 -11.571  78.437]
[ 20.052 -12.312  77.803]
[ 19.767 -10.715  79.32 ]
[ 21.207 -10.64   79.602]
[ 21.634 -11.9

[29.569 -6.136 74.601]
[30.153 -7.181 74.327]
[30.862 -3.998 74.414]
[31.874 -4.757 75.263]
[33.229 -4.067 75.329]
[34.262 -4.777 74.458]
[35.051 -5.805 75.198]
[28.733 -6.045 75.629]
[28.337 -7.178 76.46 ]
[27.763 -8.31  75.63 ]
[28.109 -9.482 75.767]
[27.329 -6.695 77.488]
[26.651 -7.708 78.38 ]
[26.072 -7.214 79.694]
[26.461 -5.897 80.112]
[25.878 -4.809 80.532]
[24.576 -4.724 80.72 ]
[26.639 -3.745 80.8  ]
[26.835 -7.97  74.726]
[26.209 -9.034 73.941]
[27.04  -9.507 72.762]
[ 26.988 -10.675  72.395]
[24.832 -8.537 73.468]
[23.774 -8.621 74.566]
[23.669 -7.62  75.51 ]
[22.891 -9.69  74.638]
[22.711 -7.678 76.5  ]
[21.929 -9.748 75.626]
[21.826 -8.74  76.554]
[27.8   -8.625 72.139]
[28.743 -9.069 71.099]
[29.827 -9.966 71.689]
[ 30.251 -10.952  71.079]
[29.342 -7.854 70.402]
[28.53  -7.294 69.239]
[29.208 -6.028 68.739]
[28.556 -5.43  67.503]
[29.492 -4.442 66.87 ]
[30.297 -9.666 72.896]
[ 31.259 -10.523  73.567]
[ 30.674 -11.901  73.876]
[ 31.44  -12.86   73.972]
[31.753 -9.866 74.8

[-11.606 -25.114  93.992]
[-10.324 -25.205  94.469]
[ -9.205 -25.469  96.616]
[ -9.116 -25.17   93.803]
[ -7.998 -25.432  95.943]
[ -7.964 -25.286  94.545]
[-14.326 -27.148  99.488]
[-14.518 -27.379 100.922]
[-13.2   -27.209 101.655]
[-12.115 -27.112 101.086]
[-15.093 -28.772 101.183]
[-14.145 -29.705 100.706]
[-16.279 -29.076 100.292]
[-13.319 -27.161 102.972]
[-12.118 -26.934 103.787]
[-11.141 -28.067 103.547]
[ -9.929 -27.865 103.472]
[-12.556 -26.762 105.235]
[-13.213 -25.447 105.576]
[-12.892 -24.396 104.987]
[-14.086 -25.419 106.479]
[-11.667 -29.283 103.38 ]
[-10.83  -30.471 103.257]
[-10.022 -30.555 101.971]
[ -8.835 -30.88  101.932]
[-11.783 -31.683 103.418]
[-12.286 -31.823 104.852]
[-13.44  -30.929 105.245]
[-14.127 -30.282 104.42 ]
[-13.693 -30.859 106.475]
[-10.6   -30.249 100.844]
[-10.174 -30.106  99.482]
[ -9.057 -29.044  99.527]
[ -7.96  -29.29   99.061]
[-11.269 -29.689  98.491]
[-12.498 -30.551  98.433]
[-13.644 -30.252  97.507]
[-14.068 -29.102  97.235]
[-14.202 -31

...

[ 4.267 24.966 86.376]
[ 2.191 26.473 85.309]
[ 3.629 24.56  85.214]
[ 2.576 25.296 84.702]
[ 4.331 24.762 90.742]
[ 5.199 23.944 91.594]
[ 6.62  23.99  91.043]
[ 6.762 24.013 89.813]
[ 4.655 22.524 91.431]
[ 3.992 22.556 90.081]
[ 3.379 23.938 90.012]
[ 7.61  23.979 91.915]
[ 9.006 23.957 91.495]
[ 9.448 22.596 90.964]
[10.294 22.495 90.08 ]
[ 9.877 24.34  92.689]
[10.25  25.812 92.739]
[11.376 26.141 93.698]
[11.258 27.082 94.496]
[12.486 25.403 93.619]
[ 8.861 21.52  91.469]
[ 9.197 20.188 90.968]
[ 8.801 19.997 89.52 ]
[ 7.629 20.162 89.155]
[ 8.504 19.134 91.852]
[ 8.902 17.719 91.468]
[ 8.291 17.135 90.554]
[ 9.825 17.179 92.087]
[ 9.774 19.64  88.679]
[ 9.501 19.574 87.247]
[ 8.466 18.526 86.88 ]
[ 7.74  18.706 85.904]
[10.813 19.377 86.472]
[11.754 20.565 86.699]
[12.999 20.442 85.824]
[13.005 19.554 84.951]
[13.921 21.248 86.056]
[ 8.341 17.438 87.632]
[ 7.373 16.412 87.286]
[ 5.962 16.909 87.59 ]
[ 5.041 16.718 86.792]
[ 7.682 15.111 88.033]
[ 9.04  14.734 87.798]
[ 5.79  17.

[-24.925  19.146  80.699]
[-24.807  20.348  80.485]
[-23.062  17.531  80.957]
[-22.318  18.653  81.437]
[-25.801  18.682  81.594]
[-26.535  19.678  82.373]
[-25.575  20.543  83.167]
[-25.734  21.762  83.239]
[-27.565  19.063  83.331]
[-28.198  20.152  84.208]
[-29.199  20.975  83.413]
[-28.634  22.191  82.724]
[-29.134  23.494  83.219]
[-24.588  19.877  83.753]
[-23.565  20.614  84.498]
[-22.955  21.728  83.661]
[-22.846  22.894  84.081]
[-22.511  19.611  84.964]
[-22.95   18.658  86.046]
[-23.833  17.493  85.682]
[-24.337  17.314  84.555]
[-24.064  16.67   86.61 ]
[-22.537  21.417  82.447]
[-21.94   22.394  81.539]
[-22.946  23.482  81.167]
[-22.623  24.68   81.189]
[-21.401  21.751  80.261]
[-20.702  22.683  79.266]
[-19.521  23.415  79.912]
[-20.235  21.94   78.022]
[-24.155  23.058  80.81 ]
[-25.214  23.987  80.419]
[-25.455  25.056  81.483]
[-25.599  26.256  81.215]
[-26.492  23.187  80.144]
[-27.61   23.944  79.442]
[-28.933  23.197  79.553]
[-28.898  21.955  79.69 ]
[-29.999  23

[-10.234  49.291  90.84 ]
[-10.763  49.494  92.196]
[-10.155  48.616  93.275]
[-9.951 48.983 94.441]
[-12.258  49.271  92.155]
[-9.832 47.384 92.927]
[-9.265 46.41  93.821]
[-10.25   46.018  94.914]
[-9.789 45.628 95.991]
[-11.553  46.141  94.653]
[-12.565  45.84   95.636]
[-13.084  44.41   95.543]
[-13.216  43.738  96.568]
[-13.783  46.778  95.515]
[-13.473  48.232  95.837]
[-14.644  49.106  95.405]
[-14.483  50.552  95.847]
[-15.8    51.261  95.912]
[-13.373  43.973  94.326]
[-13.834  42.611  94.097]
[-12.752  41.621  94.537]
[-11.566  41.859  94.275]
[-14.207  42.383  92.634]
[-14.829  41.031  92.335]
[-13.645  39.739  91.891]
[-13.405  40.051  90.162]
[-13.193  40.556  95.201]
[-12.359  39.482  95.715]
[-12.868  38.133  95.216]
[-14.04   37.789  95.451]
[-12.315  39.513  97.247]
[-11.186  38.781  97.946]
[-11.62   38.508  99.698]
[-11.15   40.111 100.365]
[-12.007  37.399  94.524]
[-12.342  36.05   94.069]
[-11.875  35.002  95.06 ]
[-10.919  35.2    95.82 ]
[-11.478  35.874  92.809

[-7.843 30.679 99.626]
[-10.418  30.797 101.396]
[-11.031  29.737 102.275]
[-12.779  29.531 101.844]
[-13.232  28.427 103.19 ]
[ -8.417  32.796 100.186]
[-7.888 33.381 98.949]
[-6.384 33.212 98.868]
[-5.764 32.817 97.891]
[-8.241 34.878 98.88 ]
[-7.658 35.534 97.63 ]
[-9.75  35.059 98.905]
[-5.733 33.542 99.988]
[ -4.293  33.392 100.086]
[-3.929 31.93  99.925]
[-3.014 31.583 99.211]
[ -3.855  33.946 101.459]
[ -3.934  35.465 101.57 ]
[ -2.484  33.407 101.818]
[ -3.965  35.922 103.023]
[ -4.712  31.042 100.573]
[ -4.446  29.607 100.387]
[-4.564 29.22  98.919]
[-3.771 28.436 98.386]
[ -5.353  28.731 101.247]
[ -5.187  28.888 102.737]
[ -4.181  29.526 103.103]
[ -6.01   28.386 103.55 ]
[-5.553 29.776 98.221]
[-5.737 29.427 96.822]
[-4.603 29.979 95.97 ]
[-4.245 29.366 94.966]
[-7.082 29.902 96.292]
[-8.267 29.12  96.814]
[-8.063 28.013 97.35 ]
[-9.405 29.628 96.701]
[-4.05  31.126 96.351]
[-2.963 31.743 95.596]
[-1.732 30.834 95.639]
[-0.985 30.727 94.668]
[-2.681 33.139 96.143]
[-3.768 3

[-25.337  44.993  92.014]
[-1.297 34.487 51.933]
[20.337 29.694 64.56 ]
[-1.754 29.348 56.543]
[-13.68   16.508  66.551]
[-19.493  52.429  67.758]
[-21.263  52.608  87.963]
[17.485 48.074 62.786]
[ 7.774 55.669 51.038]
[-24.68   27.868  74.21 ]
[-10.777  44.594  55.409]
[ 5.345 25.893 57.975]
[-25.49   37.409  75.798]
[-28.049  44.404  67.154]
[-10.146  20.096 103.947]
[-28.125  41.583  82.535]
[10.89  15.485 85.802]
[ 9.612 33.219 52.65 ]
[18.018 53.151 66.971]
[ 9.956 39.913 50.571]
[23.36  31.741 77.384]
[-16.554  37.603 103.22 ]
[-17.791  37.08   64.036]
[-15.644  15.165  69.413]
[-24.043  23.587  92.857]
[-9.746 22.71  68.386]
[-1.374 58.764 73.738]
[15.914 19.155 74.158]
[24.851 37.667 68.62 ]
[19.922 29.259 78.618]
[-24.355  51.41   71.135]
[-1.085 61.724 70.087]
[ 7.184 52.483 83.801]
[-1.696 58.59  86.087]
[-9.802 42.225 54.896]
[-14.973  14.261  75.462]
[24.55  41.657 69.52 ]
[ 5.018 60.937 70.606]
[-18.827  26.85   71.423]
[-18.439  22.037 108.597]
[-18.018  53.421  62.134]
